In [1]:
link = "https://www.reuters.com/resources/archive/us/{}{}{}.html"
reuters_db = '/home/ostapkharysh/Documents/bt_data/DB/Reuters'

In [2]:
from bs4 import BeautifulSoup as BS
import requests
from requests.exceptions import HTTPError
#import re
import datetime
#import pytz
from newspaper import Article
from db_manager import *
import sqlite3
from tqdm import tqdm


years = ['2013']#, '2014', '2015', '2016', '2017']
months = ['01',]# '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = ['01', ]#'02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'
       #'20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    
    

In [5]:
conn = create_connection(reuters_db)

for y in years:
    for m in months:
        for d in days:
            try:
                datetime.datetime(year=int(y),month=int(m),day=int(d))
            except ValueError:
                print("Day doesn't exist")
                continue
            try:
                r = requests.get(link.format(y, m, d))
                r.raise_for_status() # check if page found
            except HTTPError:
                print("No page found by this link!",r.status)
                continue
                    
            soup = BS(r.content, 'html.parser')
            span_list = soup.find_all('div' , attrs={'class':"headlineMed"})

            header = str(soup.find_all('title')).split()
            day, month, year = header[6], header[7], header[8]

            k = [[str(el).split('>')[1].split("=")[1].replace('"', ''), str(el).split()[-2]] for el in span_list
                 if not str(el).split('>')[1].split("=")[1].replace('"', '').startswith('http://www.reuters.com/news/video')]
            
            print(len(k))
            
            for link_time in k:
                try:
                    r = requests.get(link_time[0])
                    r.raise_for_status()
                    
                    article = Article(link_time[0].strip())
                    article.download()
                    article.parse()
                    title = article.title
                    text = article.text
                    time = link_time[1]
                    date = datetime.datetime.strptime("{} {} {} {}".format(month, day, year, time), '%b %d %Y %I:%M%p')

                    #print(str(date)+'\n'+title+'\n'+text+'\n'+link_time[0])
                    print(str(date)+'\n'+title+'\n'+link_time[0])
                 
                    with conn:
                        news = (str(date), title, text, link_time[0])
                        create_task(conn, news)
                except HTTPError:
                    print("No article found by this link!", article.url)
                    
                    continue
                print()
                
                
close_db(conn)

259
2013-01-01 23:52:00
Factbox: Key points in bill passed by Congress to avert "fiscal cliff"
http://www.reuters.com/article/2013/01/02/us-usa-fiscal-deal-idUSBRE90103820130102

2013-01-01 23:51:00
After fiscal win, Obama warns Congress on debt fight
http://www.reuters.com/article/2013/01/02/us-usa-fiscal-obama-idUSBRE8BT04S20130102

2013-01-01 23:51:00
Obama resumes Hawaii vacation after "fiscal cliff" bill passes
http://www.reuters.com/article/2013/01/02/us-usa-fiscal-obama-hawaii-idUSBRE90103220130102

2013-01-01 23:49:00
Obama resumes Hawaii vacation after 'fiscal cliff' bill passes
http://www.reuters.com/article/2013/01/02/usa-fiscal-obama-hawaii-idUSL1E9C207B20130102

2013-01-01 23:44:00
Indonesia Nov foreign tourist arrivals rise 5.94 pct y/y
http://www.reuters.com/article/2013/01/02/indonesia-economy-tourist-idUSJ9N09L00G20130102

2013-01-01 23:42:00
After fiscal win, Obama warns Congress on debt fight
http://www.reuters.com/article/2013/01/02/usa-fiscal-obama-idUSL1E9C2073201

No article found by this link! http://www.reuters.com/article/2013/01/02/fueloil-korea-wp-idUSL4N0A703520130102
2013-01-01 19:54:00
Singapore Q4 private home prices rise 1.8 pct q/q
http://www.reuters.com/article/2013/01/02/singapore-property-idUSS7E8EQ00X20130102

2013-01-01 19:54:00
PHILIPPINES PRESS-San Miguel, JG Summit not allowed to bid for Cebu airport - Inquirer
http://www.reuters.com/article/2013/01/02/philippines-press-idUSL4N0A703620130102

2013-01-01 19:47:00
Seoul shares seen higher as 2013 opens; fiscal cliff averted
http://www.reuters.com/article/2013/01/02/markets-korea-stocks-idUSL4N0A226E20130102

2013-01-01 19:45:00
PRECIOUS-Gold eases, focus on U.S. fiscal talks
http://www.reuters.com/article/2013/01/02/markets-precious-idUSL4N0A701U20130102

2013-01-01 19:40:00
Philippines - Market factors to watch on Jan 2
http://www.reuters.com/article/2013/01/02/markets-philippines-stocks-idUSL4N0A21U120130102

2013-01-01 19:38:00
Pennsylvania governor to sue NCAA over Penn Stat


2013-01-01 16:35:00
UPDATE 1-US Senate's 'fiscal cliff' bill packed with sweeteners
http://www.reuters.com/article/2013/01/01/usa-fiscal-extras-idUSL1E9C123Q20130101

2013-01-01 16:32:00
Editor’s Choice
http://www.reuters.com/article/2013/01/01/idUS80942018020130101

No article found by this link! http://www.reuters.com/article/2013/01/01/idUS80942018020130101
2013-01-01 16:25:00
Factbox: Key points in the Senate's deal to avert "fiscal cliff"
http://www.reuters.com/article/2013/01/01/us-usa-fiscal-deal-idUSBRE9000A220130101

2013-01-01 16:22:00
Nigerian army says 14 die in gun battle with Boko Haram
http://www.reuters.com/article/2013/01/01/us-nigeria-violence-idUSBRE90009T20130101

No article found by this link! http://www.reuters.com/article/2013/01/01/us-nigeria-violence-idUSBRE90009T20130101
2013-01-01 16:10:00
At least 61 crushed to death in Ivory Coast stampede
http://www.reuters.com/article/2013/01/01/us-ivorycoast-stadium-idUSBRE90003Z20130101

2013-01-01 16:10:00
New Year sw


2013-01-01 11:44:00
Bomb kills two, injures 50 in Pakistani city of Karachi
http://www.reuters.com/article/2013/01/01/us-pakistan-bomb-idUSBRE90005320130101

2013-01-01 11:26:00
Egypt satirist faces probe for insulting president
http://www.reuters.com/article/2013/01/01/egypt-satirist-idUSL5E9C12BN20130101

2013-01-01 11:25:00
Biden will discuss "fiscal cliff" deal with House Democrats
http://www.reuters.com/article/2013/01/01/us-usa-fiscal-biden-idUSBRE90006G20130101

No article found by this link! http://www.reuters.com/article/2013/01/01/us-usa-fiscal-biden-idUSBRE90006G20130101
2013-01-01 11:24:00
Mood drug no help for smoking cessation in prison study
http://www.reuters.com/article/2013/01/01/us-mood-drug-idUSBRE90007020130101

2013-01-01 11:22:00
$100M claim against Connecticut in school shooting is dropped
http://www.reuters.com/article/2013/01/01/usa-schoolshooting-legal-idUSL1E9C10HH20130101

2013-01-01 11:15:00
UPDATE 1-Biden will discuss 'fiscal cliff' deal with House Democ


No article found by this link! http://www.reuters.com/article/2013/01/01/cricket-england-martin-jenkins-idUSL4N0A609Y20130101
2013-01-01 07:12:00
Drifting Shell drill ship grounds on rocks off Alaska
http://www.reuters.com/article/2013/01/01/uk-alaska-ship-idUSLNE90000520130101

2013-01-01 07:11:00
Expats who bounce cheques no longer risk prison in UAE
http://www.reuters.com/article/2013/01/01/uk-ae-defaulters-idUSLNE90000220130101

No article found by this link! http://www.reuters.com/article/2013/01/01/uk-ae-defaulters-idUSLNE90000220130101
2013-01-01 06:47:00
SNAPSHOT-India stocks, bonds, rupee, swaps, call at close
http://www.reuters.com/article/2013/01/01/india-snapshot-idUSL4N0A609920130101

2013-01-01 06:44:00
Top Afghan negotiator optimistic over peace prospects
http://www.reuters.com/article/2013/01/01/us-afghanistan-reconciliation-idUSBRE90002720130101

No article found by this link! http://www.reuters.com/article/2013/01/01/us-afghanistan-reconciliation-idUSBRE9000272013010